In [1]:
import furniture_bench  # noqa: F401

from src.behavior.base import Actor
from src.eval.load_model import load_bc_actor

import wandb
import numpy as np
import torch

from omegaconf import OmegaConf

from furniture_bench.envs.observation import DEFAULT_STATE_OBS
import hydra
from src.gym import turn_off_april_tags
from src.gym.env_rl_wrapper import ResidualPolicyEnvWrapper
from furniture_bench.envs.furniture_rl_sim_env import FurnitureRLSimEnv
from src.models.residual import ResidualPolicy

from tqdm import trange


turn_off_april_tags()

api = wandb.Api()

device = "cuda" if torch.cuda.is_available() else "cpu"

Importing module 'gym_38' (/data/scratch/ankile/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /data/scratch/ankile/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.0+cu121
Device count 1
/data/scratch/ankile/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /data/scratch/ankile/.cache as PyTorch extensions root...
Emitting ninja build file /data/scratch/ankile/.cache/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


In [2]:
residual_run = api.run("residual-ppo-dr-1/cclmq6hq")
# residual_run = api.run("residual-ppo-2/runs/3iom50to")

cfg = OmegaConf.create(
    {
        **residual_run.config,
        "env": {"randomness": "low"},
        # "base_bc_poliy": "ol-state-dr-1/r9wm1uo6",
    }
)

cfg

{'env': {'randomness': 'low'}, 'seed': 2509922154, 'debug': False, 'gamma': 0.999, 'wandb': {'mode': 'online', 'entity': 'ankile', 'project': 'residual-ppo-dr-1'}, 'vf_coef': 0.5, 'ent_coef': 0, 'headless': True, 'norm_adv': True, 'num_envs': 1024, 'clip_coef': 0.2, 'target_kl': 0.1, 'batch_size': 716800, 'clip_vloss': True, 'eval_first': True, 'gae_lambda': 0.95, 'action_type': 'pos', 'clip_reward': 5, 'residual_l1': 0, 'residual_l2': 0.01, 'act_rot_repr': 'rot_6d', 'lr_scheduler': {'name': 'cosine', 'warmup_steps': 8}, 'base_bc_poliy': 'ol-state-dr-1/r9wm1uo6', 'eval_interval': 5, 'max_grad_norm': 0.5, 'num_env_steps': 700, 'update_epochs': 20, 'minibatch_size': 716800, 'num_iterations': 418, 'num_minibatches': 1, 'residual_policy': {'_target_': 'src.models.residual.ResidualPolicy', 'init_logstd': -1.5, 'action_scale': 0.1, 'action_head_std': 0, 'actor_activation': 'ReLU', 'actor_num_layers': 2, 'actor_hidden_size': 256, 'critic_activation': 'ReLU', 'critic_num_layers': 2, 'critic_hi

In [3]:
env: FurnitureRLSimEnv = FurnitureRLSimEnv(
    act_rot_repr=cfg.act_rot_repr,
    action_type=cfg.action_type,
    april_tags=False,
    concat_robot_state=True,
    ctrl_mode="diffik",
    obs_keys=DEFAULT_STATE_OBS,
    furniture="one_leg",
    gpu_id=0,
    headless=True,  # cfg.headless,
    num_envs=1024,  # cfg.num_envs,
    observation_space="state",
    randomness=cfg.env.randomness,
    max_env_steps=100_000_000,
)

env.max_force_magnitude = 0.05
env.max_torque_magnitude = 0.0025

# Load the behavior cloning actor
bc_actor: Actor = load_bc_actor(cfg.base_bc_poliy)

env: ResidualPolicyEnvWrapper = ResidualPolicyEnvWrapper(
    env,
    max_env_steps=cfg.num_env_steps,
    reset_on_success=cfg.reset_on_success,
    reset_on_failure=cfg.reset_on_failure,
)
env.set_normalizer(bc_actor.normalizer)

Making DiffIK controller with pos_scalar: 1.0, rot_scalar: 1.0
Stiffness: 1000.0, Damping: 200.0
Observation keys: ['robot_state/ee_pos', 'robot_state/ee_quat', 'robot_state/ee_pos_vel', 'robot_state/ee_ori_vel', 'robot_state/gripper_width', 'parts_poses']
Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled
Using SDF cache directory '/afs/csail.mit.edu/u/a/ankile/.isaacgym/sdf_V100'
~!~!~! Loaded/Cooked SDF triangle mesh 0 @ 0xcbb27940, resolution=512, spacing=0.000317
  ~!~! Bounds:  (-0.081250, 0.081250) (-0.015685, 0.015565) (-0.081250, 0.081251)
  ~!~! Extents: (0.162500, 0.031250, 0.162501)
  ~!~! Resolution: (512, 99, 512)
~!~!~! Loaded/Cooked SDF triangle mesh 1 @ 0xcc3e56d0, resolution=512, spacing=0.000171
  ~!~! Bounds:  (-0.015000, 0.015000) (-0.056250, 0.031250) (-0.014383, 0.015618)
  ~!~! Extents: (0.030000, 0.087500, 0.030001)
  ~!~! Resolution: (176, 512, 176)
~!~!~! Loaded/Cooked SDF triangle mesh 2 @ 0xcd755050, 

/data/scratch/ankile/miniconda3/envs/rr/lib/python3.8/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Total parameters: 8.98M
normalizer: 0.00M parameters
loss_fn: 0.00M parameters
model: 8.98M parameters
./models/legendary-sunset-7/actor_chkpt_best_success_rate.pt


In [4]:

# Residual policy setup
residual_policy: ResidualPolicy = hydra.utils.instantiate(
    cfg.residual_policy,
    obs_shape=env.observation_space.shape,
    action_shape=env.action_space.shape,
)

# Load the residual policy weights
wts = [f for f in residual_run.files() if ".pt" in f.name][0]
wts.download(replace=True)

residual_policy.load_state_dict(torch.load(wts.name)["model_state_dict"])

residual_policy.to(device)

Total parameters: 0.17M
actor_mean: 0.09M parameters
critic: 0.08M parameters


ResidualPolicy(
  (actor_mean): Sequential(
    (0): Linear(in_features=68, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=False)
  )
  (critic): Sequential(
    (0): Linear(in_features=68, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [5]:
%%time

next_obs = env.reset()
bc_actor.reset()

total_reward = 0

for step in trange(0, 100):

    # Get the base normalized action
    base_naction = bc_actor.action_normalized(next_obs)

    # Process the obs for the residual policy
    next_obs = env.process_obs(next_obs)
    next_residual_obs = torch.cat([next_obs, base_naction], dim=-1)

    with torch.no_grad():
        residual_naction_samp, logprob, _, value, naction_mean = (
            residual_policy.get_action_and_value(next_residual_obs)
        )

    residual_naction = naction_mean
    naction = base_naction + residual_naction * cfg.residual_policy.action_scale

    next_obs, reward, next_done, truncated, infos = env.step(naction)

    total_reward += reward.sum()


# Calculate the success rate
total_reward / env.num_envs

100%|██████████| 100/100 [01:41<00:00,  1.01s/it]

CPU times: user 2min 12s, sys: 1min 31s, total: 3min 43s
Wall time: 1min 43s


tensor(0., device='cuda:0')

In [6]:
# Only base policy: 51%
# With ok residual: 87%
# With better residual: 95%

In [7]:
# Wall time: 37.8 s
# Wall time: 1min 43s